# Телеграм-бот

In [1]:
!pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 213 kB 33.1 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.7.1-py3-none-any.whl size=196989 sha256=c9177f64188c74ed1eb7759c0ac62bb68e7fa4726eb9f5ba8798bb9ece636ee1
  Stored in directory: /root/.cache/pip/wheels/84/72/5f/8a7b3ca1ef1250002cf3bac48e0c72c5ee5c9596a5b1295dc1
Successfully built pyTelegramBotAPI


In [9]:
import cv2
import telebot
from google.colab.patches import cv2_imshow

In [3]:
# скопируем репозиторий darknet 
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.17 MiB | 6.45 MiB/s, done.
Resolving deltas: 100% (10405/10405), done.


In [4]:
# изменим кое-какие настройки
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

/content/darknet


In [ ]:
# соберем получившееся решение, чтобы запускать файл darknet.py 
!make

In [6]:
# скачаем файл весов модели yolov4, натренированный определять 80 классов объектов 
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq" -O yolov4-csp.weights && rm -rf /tmp/cookies.txt

--2022-11-09 11:19:33--  https://docs.google.com/uc?export=download&confirm=t&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq
Resolving docs.google.com (docs.google.com)... 74.125.24.138, 74.125.24.102, 74.125.24.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.24.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/67aj3arm1mkfk37ff6g34doe7nsl7212/1667992725000/17800843676226924807/*/1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq?e=download&uuid=03fb4684-51be-4277-887e-ce02e9290134 [following]
--2022-11-09 11:19:34--  https://doc-14-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/67aj3arm1mkfk37ff6g34doe7nsl7212/1667992725000/17800843676226924807/*/1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq?e=download&uuid=03fb4684-51be-4277-887e-ce02e9290134
Resolving doc-14-84-docs.googleusercontent.com (doc-14-84-docs.googleusercontent.com)... 142.251.10.132, 2404

Я сделал отдельный файл datatoken.py для токена

In [24]:
bot = telebot.TeleBot('5532689573:AAH4xa87G9ak2nJalDinLTfdOeWtnFuba70')

In [7]:
from darknet import *
# загрузим YOLOv4 
network, class_names, class_colors = load_network("cfg/yolov4-csp.cfg", "cfg/coco.data", "yolov4-csp.weights")
width = network_width(network)
height = network_height(network)

# darknet_helper функция для детектирования объектов на изображении
def darknet_helper(img, width, height):
  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height),
                              interpolation=cv2.INTER_LINEAR)

  # пересчет координат границ объектов, чтобы можно было отобразить на кадре
  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  # запуск модели
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  return detections, width_ratio, height_ratio

In [25]:
@bot.message_handler(content_types=['photo'])
def photographio(message):
  print(message)
  file = bot.get_file(message.photo[-1].file_id)
  bot.send_message(message.from_user.id, 'Обработка')
  dfile = bot.download_file(file.file_path)
  fimg = message.photo[-1].file_id + '.png'
  with open(fimg, 'wb') as new_file:
     new_file.write(dfile)
  new_file.close()
  image = cv2.imread(fimg)
  detections, width_ratio, height_ratio = darknet_helper(image, width, height)

  for label, confidence, bbox in detections:
    left, top, right, bottom = bbox2points(bbox)
    left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
    cv2.rectangle(image, (left, top), (right, bottom), class_colors[label], 2)
    cv2.putText(image, "{} [{:.2f}]".format(label, float(confidence)),
                    (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    class_colors[label], 2)
    
  cv2.imwrite(fimg, image)
  with open(fimg,'rb') as fil:
        byte = fil.read()
  bot.send_photo(message.from_user.id,  byte)

bot.infinity_polling()



{'content_type': 'photo', 'id': 14, 'message_id': 14, 'from_user': {'id': 453971170, 'is_bot': False, 'first_name': 'Виктор', 'username': None, 'last_name': None, 'language_code': 'ru', 'can_join_groups': None, 'can_read_all_group_messages': None, 'supports_inline_queries': None, 'is_premium': None, 'added_to_attachment_menu': None}, 'date': 1667995780, 'chat': {'id': 453971170, 'type': 'private', 'title': None, 'username': None, 'first_name': 'Виктор', 'last_name': None, 'photo': None, 'bio': None, 'join_to_send_messages': None, 'join_by_request': None, 'has_private_forwards': None, 'has_restricted_voice_and_video_messages': None, 'description': None, 'invite_link': None, 'pinned_message': None, 'permissions': None, 'slow_mode_delay': None, 'message_auto_delete_time': None, 'has_protected_content': None, 'sticker_set_name': None, 'can_set_sticker_set': None, 'linked_chat_id': None, 'location': None}, 'sender_chat': None, 'forward_from': None, 'forward_from_chat': None, 'forward_from_m

2022-11-09 12:10:08,966 (__init__.py:970 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2022-11-09 12:10:08,970 (__init__.py:972 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
